In [1]:
import pandas as pd
import numpy as np
from keybert import KeyBERT

In [2]:
df = pd.read_csv('commen_sense.csv', encoding ="utf-8")

In [3]:
from keybert import KeyBERT
kw_model = KeyBERT(model='all-MiniLM-L6-v2')

In [11]:
df.Review[44]

"I've been playing roblox since I was 9 roblox was slightly disgusting than with people and now its even more fucking disgusting as I'm older and especially with voice chat. on top of that people will get you banned for 0 reason and roblox doesn't respond at least this has happened for me. I even show them proof as if they would fucking care to look. the game has become trash in that aspect and games nowadays are getting more boring. It's really getting worthless to play this game anymore. I'm 18 now and a lot of people on roblox is not safe for children even when I was little but now its worse. Some people be talking shit like sex and etc without getting banned this is how bad it is in my view. I would like to rant more but this is some."

In [13]:
kw_model.extract_keywords(df.Review[44], keyphrase_ngram_range=(1, 3), stop_words='english')

[('banned reason roblox', 0.6328),
 ('roblox slightly disgusting', 0.6313),
 ('roblox safe', 0.5746),
 ('playing roblox', 0.5722),
 ('roblox safe children', 0.5681)]

In [26]:
kw_model.extract_keywords(df.Review[44], highlight=True, stop_words='english', top_n=20)

ve been playing roblox since was roblox was slightly disgusting than with people and now its even more fucking 
disgusting as older and especially with voice chat on top of that people will get you banned for reason and roblox 
doesn respond at least this has happened for me even show them proof as if they would fucking care to look the game
has become trash in that aspect and games nowadays are getting more boring It really getting worthless to play this
game anymore 18 now and lot of people on roblox is not safe for children even when was little but now its worse 
Some people be talking shit like sex and etc without getting banned this is how bad it is in my view would like to 
rant more but this is some

[('roblox', 0.5057),
 ('banned', 0.3426),
 ('talking', 0.2499),
 ('disgusting', 0.247),
 ('chat', 0.2194),
 ('safe', 0.2156),
 ('rant', 0.2073),
 ('like', 0.1993),
 ('trash', 0.1972),
 ('anymore', 0.1932),
 ('especially', 0.1892),
 ('older', 0.1886),
 ('care', 0.1867),
 ('worse', 0.1766),
 ('play', 0.1755),
 ('nowadays', 0.173),
 ('bad', 0.1666),
 ('game', 0.1664),
 ('games', 0.1638),
 ('children', 0.1633)]